# **UB** : Save and Restore Crystal Orientation

see: https://github.com/bluesky/hklpy/issues/50

**Objectives**

1. Save the information defining the crystal orientation into the descriptor document
1. List runs that have orientation that can be restored
1. Restore crystal orientation from a given Bluesky run

------------
# Data collection

## Setup for data collection

Use a local, temporary, file-based databroker.  It will reset after each restart of the notebook.  Prepare to define the diffractometers needed here plus some items from the ophyd simulators.

In [1]:
from bluesky import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback
import bluesky.plans as bp
import bluesky.plan_stubs as bps
import bluesky.preprocessors as bpp
import databroker
import hkl
from hkl import *
import numpy as np
import pyRestTable
from ophyd import Component, Device, EpicsSignal, Signal
from ophyd.signal import AttributeSignal, ArrayAttributeSignal
from ophyd.sim import *
import pandas as pd

bec = BestEffortCallback()
bec.disable_plots()
cat = databroker.temp().v2

RE = RunEngine({})
RE.subscribe(bec)
RE.subscribe(cat.v1.insert)
RE.md["notebook"] = "tst_UB_in_descriptor_document"
RE.md["objective"] = "Demonstrate UB matrix save & restore in descriptor document of bluesky run"

-------------

## Build simulated 4-circle diffractometer

Build two 4-circles so that we can test routines that differentiate between similar diffractometers.  Use the second one to restore orientation saved from the first.

In [2]:
class Fourc(SimulatedE4CV):
    pass

fourc = Fourc("", name="fourc")
fourc.energy.put(A_KEV / 1.54)
a0 = SI_LATTICE_PARAMETER
fourc.calc.new_sample("silicon", lattice=(a0, a0, a0, 90, 90, 90))
fourc.calc.sample.compute_UB(
    fourc.calc.sample.add_reflection(4, 0, 0, (-145.451, 0, 0, 69.0966)),
    fourc.calc.sample.add_reflection(0, 4, 0, (-145.451, 0, 90, 69.0966))
)
fourc.pa()

orange = Fourc("", name="orange")
orange.pa()

===================== ===========================================================================
term                  value                                                                      
===================== ===========================================================================
diffractometer        fourc                                                                      
geometry              E4CV                                                                       
class                 Fourc                                                                      
energy (keV)          8.05092                                                                    
wavelength (angstrom) 1.54000                                                                    
calc engine           hkl                                                                        
mode                  bissector                                                                  
positions           

Build simulators for other diffractometer geometries to test code that differentiates between various possibile sources for restore of orientation information.

In [3]:
class Kappa(SimulatedK4CV):
    pass

kappa = Kappa("", name="kappa")
kappa.energy.put(A_KEV / 1.54)
a0 = SI_LATTICE_PARAMETER
kappa.calc.new_sample("silicon", lattice=(a0, a0, a0, 90, 90, 90))
kappa.calc.sample.compute_UB(
    kappa.calc.sample.add_reflection(4, 0, 0, (55.4507, 0, 90, -69.0966)), 
    kappa.calc.sample.add_reflection(0, 4, 0, (-1.5950, 134.7568, 123.3554, -69.0966))
)
kappa.pa()

===================== =================================================================================
term                  value                                                                            
===================== =================================================================================
diffractometer        kappa                                                                            
geometry              K4CV                                                                             
class                 Kappa                                                                            
energy (keV)          8.05092                                                                          
wavelength (angstrom) 1.54000                                                                          
calc engine           hkl                                                                              
mode                  bissector                                 

In [4]:
class Sixc(SimulatedE6C):
    pass

sixc = Sixc("", name="sixc")
sixc.energy.put(A_KEV / 1.54)
a0 = SI_LATTICE_PARAMETER
sixc.calc.new_sample("silicon", lattice=(a0, a0, a0, 90, 90, 90))
sixc.calc.sample.compute_UB(
    sixc.calc.sample.add_reflection(4, 0, 0, (0, -145.451, 0, 0, 0, 69.0966)),
    sixc.calc.sample.add_reflection(0, 4, 0, (0, -145.451, 90, 0, 0, 69.0966))
)
sixc.pa()

===================== ========================================================================================================
term                  value                                                                                                   
===================== ========================================================================================================
diffractometer        sixc                                                                                                    
geometry              E6C                                                                                                     
class                 Sixc                                                                                                    
energy (keV)          8.05092                                                                                                 
wavelength (angstrom) 1.54000                                                                                  

## Collect data with all the diffractometers

Show data collection with and without the orientation information.

**Tip**: To save orientation information, add the diffractometer as an additional detector.  That's all!  Works with any scan that supports multiple detectors.

In [5]:
def scan_all():
    ### count runs ###
    # this run will not save orientation information
    yield from bp.count([noisy_det])
    # this run _will_ save orientation information for fourc
    yield from bp.count([noisy_det, fourc])
    # this run _will_ save orientation information for several diffractometers
    yield from bp.count([noisy_det, fourc, orange, kappa, sixc])

    ### scan runs ###
    yield from bp.scan([noisy_det], fourc.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det, fourc], fourc.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det], kappa.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det, kappa], kappa.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det], sixc.h, 0.9, 1.1, 2)
    yield from bp.scan([noisy_det, sixc], sixc.h, 0.9, 1.1, 2)

    ### mesh runs at the (100) ###
    # first, move to the (100)
    yield from bps.mv(fourc.h, 1, fourc.k, 0, fourc.l, 0)
    yield from bp.rel_grid_scan([noisy_det], fourc.h, -0.1, 0.1, 3, fourc.k, -0.1, 0.1, 3)
    yield from bp.rel_grid_scan([noisy_det, fourc], fourc.h, -0.1, 0.1, 3, fourc.k, -0.1, 0.1, 3)

Run the scans, gather all the uids into a variable to be ignored.  That way, they do not print.

In [6]:
_uids = RE(scan_all())



Transient Scan ID: 1     Time: 2022-06-13 00:06:20
Persistent Unique Scan ID: 'd321e9b5-757b-42fe-a57b-2a9eb971aec6'
New stream: 'primary'
+-----------+------------+------------+
|   seq_num |       time |  noisy_det |
+-----------+------------+------------+
|         1 | 00:06:20.1 |      0.980 |
+-----------+------------+------------+
generator count ['d321e9b5'] (scan num: 1)





Transient Scan ID: 2     Time: 2022-06-13 00:06:20
Persistent Unique Scan ID: '88782b29-df5b-4fd7-8b89-6597f814f733'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+
|   seq_num |       time |    fourc_h |    fourc_k |    fourc_l |  noisy_det |
+-----------+------------+------------+------------+------------+------------+
|         1 | 00:06:20.2 |      0.000 |      0.000 |      0.000 |      0.953 |
+-----------+------------+------------+------------+------------+------------+
generator count ['88782b29'] (scan num: 2)





Transient Scan ID: 3     Time

------------
# Show the orientation information that was collected

Show the full contents of the descriptor document (primary stream) for the `fourc` "detector" from the run with `scan_id=5`.  This is where the orientation information is saved.  You may need to expand the *Data variables* row to see all the orientation information.

In [7]:
cat[5].primary.config["fourc"].read()

<xarray.Dataset>
Dimensions:                    (time: 2, dim_0: 6, dim_1: 6, dim_2: 3,
                                dim_3: 3, dim_4: 3, dim_5: 3, dim_6: 2,
                                dim_7: 3, dim_8: 4, dim_9: 4, dim_10: 4,
                                dim_11: 21)
Coordinates:
  * time                       (time) float64 1.655e+09 1.655e+09
Dimensions without coordinates: dim_0, dim_1, dim_2, dim_3, dim_4, dim_5,
                                dim_6, dim_7, dim_8, dim_9, dim_10, dim_11
Data variables: (12/21)
    fourc_energy               (time) float64 8.051 8.051
    fourc_energy_units         (time) <U3 'keV' 'keV'
    fourc_energy_offset        (time) int64 0 0
    fourc_geometry_name        (time) <U4 'E4CV' 'E4CV'
    fourc_class_name           (time) <U5 'Fourc' 'Fourc'
    fourc_sample_name          (time) <U7 'silicon' 'silicon'
    ...                         ...
    fourc__hklpy_version       (time) <U23 '1.0.1+10.ge87af6a.dirty' '1.0.1+1...
    fourc__pseudos             (time, dim_7) <U1 'h' 'k' 'l' 'h' 'k' 'l'
    fourc__reals               (time, dim_8) <U5 'omega' 'chi' ... 'phi' 'tth'
    fourc__constraints         (time, dim_9, dim_10) float64 -180.0 ... 1.0
    fourc__mode                (time) <U9 'bissector' 'bissector'
    fourc_orientation_attrs    (time, dim_11) <U19 'orientation_attrs' ... '_...

## Show orientation that was saved

In `scan_id=3`, orientation information from 4 different diffractometers was saved with the run.  Show what is available from each of those diffractometers.  The columns in the next table are the diffractometers, the rows are the orientation information saved for each.

In [8]:
roi = run_orientation_info(cat[3])
pd.DataFrame(roi)

,fourc,kappa,orange,sixc
energy,8.050922,8.050922,8.0,8.050922
energy_units,keV,keV,keV,keV
energy_offset,0,0,0,0
geometry_name,E4CV,K4CV,E4CV,E6C
class_name,Fourc,Kappa,Fourc,Sixc
sample_name,silicon,silicon,main,silicon
lattice,"[5.431020511, 5.431020511, 5.431020511, 90.0, ...","[5.431020511, 5.431020511, 5.431020511, 90.0, ...","[1.54, 1.54, 1.54, 90.0, 90.0, 90.0]","[5.431020511, 5.431020511, 5.431020511, 90.0, ..."
lattice_reciprocal,"[1.156906937555034, 1.156906937555034, 1.15690...","[1.156906937555034, 1.156906937555034, 1.15690...","[4.079990459207523, 4.079990459207523, 4.07999...","[1.156906937555034, 1.156906937555034, 1.15690..."
U,"[[-1.2217304763832563e-05, -0.9999999999253688...","[[-1.7453292519418068e-05, -6.226958714415449e...","[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, ...","[[-1.2217304763832563e-05, -1.221730476200898e..."
UB,"[[-1.413428463950206e-05, -1.1569069374686927,...","[[-2.0191835198892143e-05, -7.204011736576008e...","[[4.079990459207523, -2.4982736282101165e-16, ...","[[-1.413428463950206e-05, -1.4134284637392341e..."


## Show runs with orientation information

Since a given run (``scan_id``) may have more than one set of orientation information, corresponding to more than one diffractometer, report for each when found.  Here, extra columns are reported for energy & units, and the crystal lattice parameters.  The names are taken from the above table.  (They must be one of the names in the `orientation_attrs` list.)

Use this type of listing to determine which **scan_id** and **diffractometer_name** has the orientation you wish to recover.  If the ``scan_id`` is not unique, identify the run with the **uid** (as a string, such as `cat["007abcd"]`).

In [9]:
list_orientation_runs(cat, "energy", "energy_units", "lattice")

,scan_id,sample_name,diffractometer_name,geometry_name,energy,energy_units,lattice,uid
0,2,silicon,fourc,E4CV,8.050922,keV,"[5.431020511, 5.431020511, 5.431020511, 90.0, ...",88782b2
1,3,silicon,fourc,E4CV,8.050922,keV,"[5.431020511, 5.431020511, 5.431020511, 90.0, ...",40a645b
2,3,silicon,kappa,K4CV,8.050922,keV,"[5.431020511, 5.431020511, 5.431020511, 90.0, ...",40a645b
3,3,main,orange,E4CV,8.000000,keV,"[1.54, 1.54, 1.54, 90.0, 90.0, 90.0]",40a645b
4,3,silicon,sixc,E6C,8.050922,keV,"[5.431020511, 5.431020511, 5.431020511, 90.0, ...",40a645b
5,5,silicon,fourc,E4CV,8.050922,keV,"[5.431020511, 5.431020511, 5.431020511, 90.0, ...",c5e1e9b
6,7,silicon,kappa,K4CV,8.050922,keV,"[5.431020511, 5.431020511, 5.431020511, 90.0, ...",f1ae942
7,9,silicon,sixc,E6C,8.050922,keV,"[5.431020511, 5.431020511, 5.431020511, 90.0, ...",8515819
8,10,silicon,fourc,E4CV,8.050922,keV,"[5.431020511, 5.431020511, 5.431020511, 90.0, ...",c43f568
9,11,silicon,fourc,E4CV,8.050922,keV,"[5.431020511, 5.431020511, 5.431020511, 90.0, ...",ea4b3a8


------------
# Restore orientation information

This demo will restore the orientation information from a `fourc` run (choosing `scan_id=2`) to the `orange` diffractometer.  They have the same **geometry_name** so the information is compatible.

First, get the orientation information from the chosen run.  Show the information for the `fourc` diffractometer.

In [10]:
info = run_orientation_info(cat[2])

import pprint
pprint.pprint(info["fourc"])

{'U': [[-1.2217304763832563e-05, -0.9999999999253688, 0.0],
       [0.0, 0.0, 1.0],
       [-0.9999999999253688, 1.2217304763832563e-05, 0.0]],
 'UB': [[-1.413428463950206e-05, -1.1569069374686927, 7.084098436944218e-17],
        [0.0, 0.0, 1.156906937555034],
        [-1.1569069374686927, 1.41342846395729e-05, 7.083925341879798e-17]],
 '_constraints': [[-180.0, 180.0, 0.0, 1.0],
                  [-180.0, 180.0, 0.0, 1.0],
                  [-180.0, 180.0, 0.0, 1.0],
                  [-180.0, 180.0, 0.0, 1.0]],
 '_hklpy_version': '1.0.1+10.ge87af6a.dirty',
 '_mode': 'bissector',
 '_pseudos': ['h', 'k', 'l'],
 '_reals': ['omega', 'chi', 'phi', 'tth'],
 'class_name': 'Fourc',
 'diffractometer_name': 'fourc',
 'energy': 8.050921974025975,
 'energy_offset': 0,
 'energy_units': 'keV',
 'geometry_name': 'E4CV',
 'lattice': [5.431020511, 5.431020511, 5.431020511, 90.0, 90.0, 90.0],
 'lattice_reciprocal': [1.156906937555034,
                        1.156906937555034,
                        

Earlier, the sample and orientation were setup on the `fourc` with these steps:

```python
fourc.energy.put(A_KEV / 1.54)
a0 = SI_LATTICE_PARAMETER
fourc.calc.new_sample("silicon", lattice=(a0, a0, a0, 90, 90, 90))
fourc.calc.sample.compute_UB(
    fourc.calc.sample.add_reflection(4, 0, 0, (-145.451, 0, 0, 69.0966)),
    fourc.calc.sample.add_reflection(0, 4, 0, (-145.451, 0, 90, 69.0966))
)
```

We have all the information here to repeat those steps for the `orange` diffractometer (same `E4CV` geometry).

term | recovered orientation
:--- | :---
energy | `info["fourc"]["energy"]`
sample name | `info["fourc"]["sample_name"]`
sample lattice | `info["fourc"]["lattice"]`
first reflection | `info["fourc"]["reflections_details"][0]`: `["reflection"]` and `["position"]`
second reflection | `info["fourc"]["reflections_details"][1]`: `["reflection"]` and `["position"]`

The energy and sample info are ready to use.  Both the constraints and reflections info will take a bit of reformatting.

Due to issues with how bluesky writes data, the constraints info was written with all floating point numbers (including the boolean for the `fit` parameter).

1. The order of constraints is exactly the order of the `_reals` so those associations and conversions must be handled.
1. We can't just use the dictionaries for `reflection` and `position` in ``info`` since order is important and those are recovered in alphabetical order.  That's why the ordered lists for *pseudo* and *real* positioners have been saved with the orientation information.  Those lists provide the canonical order for *any* diffractometer geometry.

**Before restoring**, need to check the target diffractometer to see if it already has these settings.

In [11]:
print(f"{info['fourc']['energy'] = }")
print(f"{info['fourc']['energy_units'] = }")
print(f"{info['fourc']['energy_offset'] = }")
print(f"{orange.energy.get() = }")
print(f"{orange.energy_units.get() = }")
print(f"{orange.energy_offset.get() = }")
print()
print(f"{info['fourc']['sample_name'] = }")
print(f"{list(orange.calc._samples.keys()) = }")
print()
print(f"{orange.calc._samples = }")

info['fourc']['energy'] = 8.050921974025975
info['fourc']['energy_units'] = 'keV'
info['fourc']['energy_offset'] = 0
orange.energy.get() = 8.0
orange.energy_units.get() = 'keV'
orange.energy_offset.get() = 0

info['fourc']['sample_name'] = 'silicon'
list(orange.calc._samples.keys()) = ['main']

orange.calc._samples = {'main': HklSample(name='main', lattice=LatticeTuple(a=1.54, b=1.54, c=1.54, alpha=90.0, beta=90.0, gamma=90.0), ux=Parameter(name='None (internally: ux)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), uy=Parameter(name='None (internally: uy)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), uz=Parameter(name='None (internally: uz)', limits=(min=-180.0, max=180.0), value=0.0, fit=True, inverted=False, units='Degree'), U=array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), UB=array([[ 4.07999046e+00, -2.49827363e-16, -2.49827363e-16],
       [ 0.00000000e+00,  4.07999046e+00, -2.498273

We need to change the energy (units and offset do not need to be changed), create a new sample, define the two reflections, then compute **UB**.  Then compare that computed **UB** with the recovered value.

## Restore the recovered `fourc` orientation into `orange`

In [12]:
from hkl import restore_orientation
orange = Fourc("", name="orange")  # TODO: remove this line before publishing
restore_orientation(info["fourc"], orange)

Compare the **UB** matrices.  (Convert the recovered **UB** matrix to a numpy array to match the type in the diffractometer.  Expect `True` in all cells of the 3x3 matrix.)

In [13]:
print(f'{np.array(info["fourc"]["UB"]) = }')
print(f"{orange.UB.get() = }")
# finally, compare the two matrices cell by cell
np.array(info["fourc"]["UB"]) == orange.UB.get()

np.array(info["fourc"]["UB"]) = array([[-1.41342846e-05, -1.15690694e+00,  7.08409844e-17],
       [ 0.00000000e+00,  0.00000000e+00,  1.15690694e+00],
       [-1.15690694e+00,  1.41342846e-05,  7.08392534e-17]])
orange.UB.get() = array([[-1.41342846e-05, -1.15690694e+00,  7.08409844e-17],
       [ 0.00000000e+00,  0.00000000e+00,  1.15690694e+00],
       [-1.15690694e+00,  1.41342846e-05,  7.08392534e-17]])


array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [14]:
orange.pa()

===================== ===========================================================================
term                  value                                                                      
===================== ===========================================================================
diffractometer        orange                                                                     
geometry              E4CV                                                                       
class                 Fourc                                                                      
energy (keV)          8.05092                                                                    
wavelength (angstrom) 1.54000                                                                    
calc engine           hkl                                                                        
mode                  bissector                                                                  
positions           

## Try to restore the recovered `fourc` orientation in `kappa`

Try to share a recovered orientation from `fourc`, `scan_id=2` with the `kappa` diffractometer.

In [15]:
try:
    info = run_orientation_info(cat[2])
    restore_orientation(info["fourc"], kappa)
except ValueError as exc:
    print(f"{exc = }")

exc = ValueError('Geometries do not match: Orientation=E4CV, kappa=K4CV, will not restore.')


Restore is not be possible since the geometries are not identical.  (To avoid stopping the notebook with an exception here, we used a `try..except` clause.)

## Try to create a sample when it already exists
Then, try to restore the sample and lattice when they already exist.

In [16]:
from hkl import restore_sample

print(f"{kappa.sample_name.get() = }")
try:
    info = run_orientation_info(cat[2])
    restore_sample(info["fourc"], kappa)
    kappa.wh()
except Exception as exc:
    print(f"{exc = }")

kappa.sample_name.get() = 'silicon'
exc = ValueError("Sample 'silicon' already exists in kappa.")


## Restore the energy

In [17]:
from hkl import restore_energy

# make a new kappa, then restore the sample
kappa = Kappa("", name="kappa")
print(f"{kappa.sample_name.get() = }")
try:
    info = run_orientation_info(cat[2])
    restore_energy(info["fourc"], kappa)
    restore_sample(info["fourc"], kappa)
    kappa.wh()
except Exception as exc:
    print(f"{exc = }")

kappa.sample_name.get() = 'main'
===================== ========= =========
term                  value     axis_type
===================== ========= =========
diffractometer        kappa              
sample name           silicon            
energy (keV)          8.05092            
wavelength (angstrom) 1.54000            
calc engine           hkl                
mode                  bissector          
h                     0.0       pseudo   
k                     0.0       pseudo   
l                     0.0       pseudo   
komega                0         real     
kappa                 0         real     
kphi                  0         real     
tth                   0         real     
===================== ========= =========

